## 1. Setup Environment and Load Libraries

Import all required libraries for model loading, text processing, similarity calculations, and performance evaluation.

In [1]:
# Import required libraries
from transformers import AutoModel, AutoTokenizer, LlamaModel, LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from peft import PeftModel
import torch
import torch.nn.functional as F
import openai
import numpy as np
import os
import pandas as pd
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import requests
import time
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns
import json

print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Initialize API Keys and Environment Variables

Load environment variables from .env file and set up API keys for external services.

In [2]:
# Load environment variables from .env file
# load_dotenv()

# Get API keys from environment variables
#openai.api_key = os.getenv('OPENAI_API_KEY')

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

# Validate API keys
if openai.api_key is None:
    print("⚠️  OpenAI API key not found in .env file. Please set OPENAI_API_KEY in your .env file.")
else:
    print("✅ OpenAI API key loaded successfully.")

✅ OpenAI API key loaded successfully.


## 3. Load Embedding Models

Initialize all embedding models with proper error handling. This may take several minutes as models are downloaded and loaded.

## 4. Define Similarity Calculation Functions

Implement functions for calculating similarity scores using different embedding models.

In [3]:
def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors"""
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        return 0.0
    return float(a.dot(b) / (na * nb))

In [4]:
def get_openai_embedding(text, model="text-embedding-3-small"):
    """Gets OpenAI embedding for a given text."""
    if openai.api_key is None:
        print("OpenAI API key not set. Cannot get embedding.")
        return None
    try:
        text = text.replace("\n", " ")
        return openai.embeddings.create(input=[text], model=model).data[0].embedding
    except Exception as e:
        print(f"Error getting OpenAI embedding: {e}")
        return None

def calculate_openai_similarity(resume_text, job_text):
    """Calculates cosine similarity between resume and job text using OpenAI embeddings."""
    resume_embedding = get_openai_embedding(resume_text)
    job_embedding = get_openai_embedding(job_text)

    if resume_embedding is None or job_embedding is None:
        print("Failed to get OpenAI embeddings. Cannot calculate similarity.")
        return None

    # Calculate similarity using the cosine function
    similarity_score = cosine_similarity(resume_embedding, job_embedding)
    return similarity_score

print("✅ OpenAI similarity functions defined.")

✅ OpenAI similarity functions defined.


## 5. Load and Sample Dataset

Load the dataset and create a balanced sample for testing.

In [5]:
from google.colab import drive
drive.mount('/content/drive')
DATASET_PATH='/content/drive/MyDrive/AI-ML Self Learning/next_horizon/resume_job_recommendation/model-shashu2325-resume-job-matcher-lora'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Load the dataset
print("Loading dataset...")
df = pd.read_csv(f'{DATASET_PATH}/dataset.csv')
print(f"✅ Dataset loaded successfully. Shape: {df.shape}")

# Display basic information about the dataset
print(f"\nDataset Info:")
print(f"Total records: {len(df)}")
print(f"Columns: {list(df.columns)}")
print(f"\nDecision distribution:")
print(df['Decision'].value_counts())

# Split into select and reject categories
select_df = df[df['Decision'] == 'select']
reject_df = df[df['Decision'] == 'reject']

print(f"\nSelect records: {len(select_df)}")
print(f"Reject records: {len(reject_df)}")

# Determine sample sizes for balanced representation
total_samples = 1000  # Adjust this number based on your needs
num_select = min(len(select_df), total_samples // 2)
num_reject = min(len(reject_df), total_samples - num_select)

# Adjust if one category is much smaller
if num_select + num_reject < total_samples:
    if len(select_df) > len(reject_df):
        num_select = min(len(select_df), total_samples - num_reject)
    else:
        num_reject = min(len(reject_df), total_samples - num_select)

print(f"\nSampling {num_select} select records and {num_reject} reject records...")

# Sample the data
sampled_select_df = select_df.sample(n=num_select, random_state=42)
sampled_reject_df = reject_df.sample(n=num_reject, random_state=42)

# Combine and shuffle
sampled_df = pd.concat([sampled_select_df, sampled_reject_df])
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Final sample size: {len(sampled_df)} records")
print(f"Sample distribution: {sampled_df['Decision'].value_counts().to_dict()}")

Loading dataset...
✅ Dataset loaded successfully. Shape: (10174, 5)

Dataset Info:
Total records: 10174
Columns: ['Role', 'Resume', 'Decision', 'Reason_for_decision', 'Job_Description']

Decision distribution:
Decision
reject    5114
select    5060
Name: count, dtype: int64

Select records: 5060
Reject records: 5114

Sampling 500 select records and 500 reject records...
✅ Final sample size: 1000 records
Sample distribution: {'reject': 500, 'select': 500}


## 6. Calculate Similarity Scores for All Models

Process each resume-job pair through all available embedding models. This may take several minutes depending on the sample size and available models.

In [12]:
# Initialize score lists for all models
openai_scores = []

print("🚀 Starting similarity calculation for all models...")
print(f"Processing {len(sampled_df)} resume-job pairs...")

for index, row in sampled_df.iterrows():
    resume_text = str(row['Resume']) if pd.notna(row['Resume']) else ""
    job_text = str(row['Job_Description']) if pd.notna(row['Job_Description']) else ""

    print(f"\n📋 Processing row {index+1}/{len(sampled_df)}")

    # Calculate OpenAI similarity
    try:
        if openai.api_key is None:
            print(f"  ⏭️  Skipping OpenAI similarity (API key not set)")
            openai_score = None
        else:
            print(f"  🔍 Calculating OpenAI similarity...")
            openai_score = calculate_openai_similarity(resume_text, job_text)
            print(f"  ✅ OpenAI similarity: {openai_score}")
    except Exception as e:
        print(f"  ❌ Error calculating OpenAI similarity: {e}")
        openai_score = None
    openai_scores.append(openai_score)

print("\n🎉 Score calculation complete!")

🚀 Starting similarity calculation for all models...
Processing 1000 resume-job pairs...

📋 Processing row 1/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI similarity: 0.6665256834471475

📋 Processing row 2/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI similarity: 0.4304944558833671

📋 Processing row 3/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI similarity: 0.5265663025480486

📋 Processing row 4/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI similarity: 0.5287259579597844

📋 Processing row 5/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI similarity: 0.3744969984247195

📋 Processing row 6/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI similarity: 0.42263386073659237

📋 Processing row 7/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI similarity: 0.40119943032067146

📋 Processing row 8/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI similarity: 0.38852222762430166

📋 Processing row 9/1000
  🔍 Calculating OpenAI similarity...
  ✅ OpenAI simi

## 7. Generate Predictions and Classification

Convert similarity scores to binary predictions using a threshold and create a comprehensive comparison table.

In [24]:
# Add the scores as new columns to the sampled DataFrame
sampled_df['openai_similarity'] = openai_scores

# Define threshold for classification
threshold = 0.3
print(f"Using threshold: {threshold} for classification")

# Create predictions for all models
def make_prediction(score, threshold=0.5):
    if score is None or pd.isna(score):
        return 'unknown'
    return 'select' if score > threshold else 'reject'

sampled_df['openai_prediction'] = sampled_df['openai_similarity'].apply(lambda x: make_prediction(x, threshold))

print("✅ Predictions generated for all models!")

# Display the comparative table
print("\n📊 Comparative Table of Similarity Scores and Predictions:")
display_columns = ['Role', 'Decision',
                  'openai_similarity', 'openai_prediction']

comparison_table = sampled_df[display_columns]
print(comparison_table.to_string(index=False, float_format='%.4f'))

Using threshold: 0.3 for classification
✅ Predictions generated for all models!

📊 Comparative Table of Similarity Scores and Predictions:
                        Role Decision  openai_similarity openai_prediction
               Data Engineer   reject             0.6665            select
      Database Administrator   reject             0.4305            select
       E-commerce Specialist   reject             0.5266            select
           Software Engineer   reject             0.5287            select
       IT Support Specialist   select             0.3745            select
             Product Manager   reject             0.4226            select
              Game Developer   reject             0.4012            select
              data scientist   reject             0.3885            select
             Cloud Architect   reject             0.5060            select
                Data Analyst   select             0.5618            select
             DevOps Engineer   rejec

## 8. Evaluate Model Performance

Calculate comprehensive performance metrics for each embedding model including accuracy, precision, recall, and F1-score.

In [25]:
# Define models to evaluate
models = {
    'OpenAI': ('openai_prediction', 'openai_similarity')
}

# Store results for comparison
results_summary = []

print("📊 COMPREHENSIVE MODEL PERFORMANCE EVALUATION")
print("=" * 80)

for model_name, (pred_col, sim_col) in models.items():
    print(f"\n🔍 {model_name} Model Performance:")
    print("-" * 50)

    # Filter out rows with unknown predictions
    valid_mask = sampled_df[pred_col] != 'unknown'
    valid_df = sampled_df[valid_mask]

    if len(valid_df) == 0:
        print(f"❌ No valid predictions for {model_name}")
        results_summary.append({
            'Model': model_name,
            'Accuracy': None,
            'Precision': None,
            'Recall': None,
            'F1': None,
            'Valid_Samples': 0
        })
        continue

    try:
        # Calculate metrics
        accuracy = accuracy_score(valid_df['Decision'], valid_df[pred_col])
        precision, recall, f1, _ = precision_recall_fscore_support(
            valid_df['Decision'], valid_df[pred_col], average='weighted'
        )

        print(f"✅ Accuracy: {accuracy:.4f}")
        print(f"✅ Precision: {precision:.4f}")
        print(f"✅ Recall: {recall:.4f}")
        print(f"✅ F1-Score: {f1:.4f}")
        print(f"📈 Valid Samples: {len(valid_df)}/{len(sampled_df)}")

        # Detailed classification report
        print(f"\n📋 Detailed Classification Report:")
        print(classification_report(valid_df['Decision'], valid_df[pred_col]))

        # Store results
        results_summary.append({
            'Model': model_name,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1': f1,
            'Valid_Samples': len(valid_df)
        })

    except Exception as e:
        print(f"❌ Error calculating metrics for {model_name}: {e}")
        results_summary.append({
            'Model': model_name,
            'Accuracy': None,
            'Precision': None,
            'Recall': None,
            'F1': None,
            'Valid_Samples': len(valid_df)
        })

print("\n🎉 Performance evaluation complete!")

📊 COMPREHENSIVE MODEL PERFORMANCE EVALUATION

🔍 OpenAI Model Performance:
--------------------------------------------------
✅ Accuracy: 0.4970
✅ Precision: 0.4809
✅ Recall: 0.4970
✅ F1-Score: 0.3627
📈 Valid Samples: 1000/1000

📋 Detailed Classification Report:
              precision    recall  f1-score   support

      reject       0.46      0.04      0.07       500
      select       0.50      0.96      0.66       500

    accuracy                           0.50      1000
   macro avg       0.48      0.50      0.36      1000
weighted avg       0.48      0.50      0.36      1000


🎉 Performance evaluation complete!


## 9. Create Summary Comparison Table

Generate a comprehensive summary table ranking all models by performance metrics.

In [26]:
# Create summary comparison table
print("🏆 SUMMARY COMPARISON TABLE")
print("=" * 80)

results_df = pd.DataFrame(results_summary)
print(results_df.to_string(index=False, float_format='%.4f'))

# Find best performing model
valid_results = results_df[results_df['Accuracy'].notna()]
if len(valid_results) > 0:
    best_model = valid_results.loc[valid_results['Accuracy'].idxmax()]
    print(f"\n🥇 Best Performing Model: {best_model['Model']} (Accuracy: {best_model['Accuracy']:.4f})")

    # Sort by accuracy for ranking
    sorted_results = valid_results.sort_values('Accuracy', ascending=False)
    print(f"\n📊 Model Ranking by Accuracy:")
    for i, (_, row) in enumerate(sorted_results.iterrows(), 1):
        medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else f"{i}."
        print(f"{medal} {row['Model']}: {row['Accuracy']:.4f}")
else:
    print("\n⚠️  No valid results found for comparison.")

print("\n" + "=" * 80)

🏆 SUMMARY COMPARISON TABLE
 Model  Accuracy  Precision  Recall     F1  Valid_Samples
OpenAI    0.4970     0.4809  0.4970 0.3627           1000

🥇 Best Performing Model: OpenAI (Accuracy: 0.4970)

📊 Model Ranking by Accuracy:
🥇 OpenAI: 0.4970

